#### The usual imports

In [1]:
%matplotlib inline
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interactive, interact, fixed

##### A plotting method that will be very helpful down the line

In [2]:
def _plot_over_time_3d(sequence: np.array, fig, ax, n: int):
    assert isinstance(n, int)
    ax.clear()
    ax.scatter(sequence[:n, 0], sequence[:n, 1], sequence[:n, 2])
    display(fig.canvas)
def plot_over_time_3d(sequence: np.array):
    plt.ioff();fig = plt.figure(); plt.ion()
    ax = fig.add_subplot(111, projection='3d')
    return interact(_plot_over_time_3d, sequence=fixed(sequence), fig=fixed(fig), ax=fixed(ax), n=(0, len(sequence)))

## Initial Observation
This began with a though about Z-scoring data. Given a set of datapoints $\mathbf{x} = [x_1, x_2, ..., x_n]$, applying a Z-score ($z_i = \frac{x_i - \mu}{\sigma}$ where $\mu = \frac{1}{n}\sum_{i=1}^{n} x_i$) can be viewed as a function $Z: \mathbb{R}^n \rightarrow \mathbb{R}^n$. In that light, what happens if we apply $Z\circ Z\circ ... \circ Z (\mathbf{x}) = Z^{(k)}(\mathbf{x})$?

In [4]:
n, k = 3, 10
seed = np.random.randn(n)
sequence = [seed]
for _ in range(k):
    sequence.append(zscore(sequence[-1]))
sequence = np.stack(sequence)
plot_over_time_3d(sequence);

interactive(children=(IntSlider(value=5, description='n', max=11), Output()), _dom_classes=('widget-interact',…

Now, this is not particularly suprising (that we only get two points), since if $\mathbf{z} = Z(\mathbf{x})$, then $\mu_{\mathbf{z}} = 0$ and $\sigma_{\mathbf{z}} = 1 \rightarrow Z(\mathbf{z}) = \mathbf{z}$.

However, what happens if instead we say $Z'(\mathbf{x}) = |\frac{\mathbf{x}-\mu}{\sigma}|$? This has a real-world interpretation: the **magnitude** of the deviation from the mean (when we don't care about the direction of deviation, just how large it is)

In [16]:
n, k = 3, 500
seed = np.random.randn(n)
sequence = [seed]
Z = lambda x: np.abs(zscore(x))
for _ in range(k):
    sequence.append(Z(sequence[-1]))
sequence = np.stack(sequence)
plot_over_time_3d(sequence);

interactive(children=(IntSlider(value=250, description='n', max=501), Output()), _dom_classes=('widget-interac…

Now we see slowly but surely, a definite pattern begins to emerge! In fact it appears as though all the points (with the exception of the seed point) in our sequence are constrained to some kind of spherical object. Why is this? Let's observe the relationship between the mean and the variance of each of the points in our sequence:

In [17]:
index = list(range(sequence.shape[0]))
means = np.mean(sequence, axis=1)
variances = np.var(sequence, axis=1)
fig, ax = plt.subplots()
ax.scatter(index, means, label='Mean')
ax.scatter(index, variances, label='Variance')
ax.scatter(index, means**2 + variances, label='Mean^2 + Variance')
ax.set_xlabel('Sequence Index');ax.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As we can see, the $\mu^{2} + \sigma^{2} =1$ for all points in the sequence outside of the seed. And this actually makes a lot of sense because if we consider $Z(\mathbf{x})$ as the values for a categorical random variable $\Xi$ with uniform PMF, then we get that $\Xi$ has mean 0 and variance 1 ($\rightarrow \mathbf{E}(\Xi^2)=1$). When we take the absolute value, we will get that the mean and variance change, however $\mathbf{E}(\Xi^2)$ will remain the same! So by the definition of variance: $Var(\Xi) = \mathbf{E}(\Xi^2)-\mathbf{E}(\Xi)^2 \rightarrow 1 = Var(\Xi) + \mathbf{E}(\Xi)^2$ 

In n-dimesnions, it is not too hard to see that in a Euclidean sense, this is sample from a shifted/scaled version of the n-dimensions hypersphere. I think this is more interesting, however, because of the potential to be sampling from a constant-error hypothesis space (using the fact that decomposition of Generalized Error).